In [1]:
import re, math, json
from pathlib import Path
import numpy as np
import pandas as pd
import rasterio
import matplotlib.pyplot as plt


### Configuración

In [2]:
BASE = Path("descargas_sentinelhub")  # carpeta de los TIF
LAKES = ["atitlan", "amatitlan"]      # subcarpetas
BANDS_SUFFIX = ".bands.tiff"    # 9 bandas: B02,B03,B04,B05,B07,B08,B8A,B11,B12 (float32)
MASK_SUFFIX  = ".mask.tiff"           # 2 bandas: SCL, dataMask (uint8)
SCL_BAD = {3,8,9,10,11}

# Umbrales del script CyanoLakes
MNDWI_thr = 0.42
NDWI_thr  = 0.4
FILTER_UABS = True

### Funciones de utilidad

In [3]:
def list_date_bases(lake_folder: Path):
    """
    Devuelve una lista de paths 'base' por fecha para un lago:
    BASE/lake/lake__YYYY-MM-DD  (sin sufijo)
    """
    bases = []
    for tif in lake_folder.glob(f"*__20??-??-??{BANDS_SUFFIX}"):
        date = re.search(r"__(\d{4}-\d{2}-\d{2})\.bands\.tiff$", tif.name).group(1)
        base = tif.with_suffix("").with_suffix("")
        bases.append((date, base))
    return sorted(bases, key=lambda x: x[0])

def load_stack(base_path: Path):
    """Lee el stack de 9 bandas (float32 reflectancia) y la máscara (SCL, dataMask)."""
    bands_path = base_path.with_suffix(BANDS_SUFFIX)
    mask_path  = base_path.with_suffix(MASK_SUFFIX)

    with rasterio.open(bands_path) as src:
        stack = src.read().astype("float32")
        b02,b03,b04,b05,b07,b08,b8a,b11,b12 = stack

    with rasterio.open(mask_path) as msrc:
        m = msrc.read()
        scl, dm = m[0].astype("int16"), m[1].astype("int16")

    return (b02,b03,b04,b05,b07,b08,b8a,b11,b12), scl, dm

### Índices calculados del script de Cyanobacteria

In [4]:

def water_mask(b02,b03,b04,b08,b11,b12):
    ndvi  = (b08 - b04) / (b08 + b04 + 1e-6)                                     # (ndvi)
    mndwi = (b03 - b11) / (b03 + b11 + 1e-6)                                     # (mndwi)
    ndwi  = (b03 - b08) / (b03 + b08 + 1e-6)                                     # (ndwi)
    ndwi_leaves = (b08 - b11) / (b08 + b11 + 1e-6)                               # (ndwi_leaves)
    aweish  = b02 + 2.5*b03 - 1.5*(b08 + b11) - 0.25*b12                         # (aweish)
    aweinsh = 4*(b03 - b11) - (0.25*b08 + 2.75*b11)                              # (aweinsh)
    dbsi = ((b11 - b03) / (b11 + b03 + 1e-6)) - ndvi                             # (dbsi)

    ws = ((mndwi > MNDWI_thr) | (ndwi > NDWI_thr) | (aweinsh > 0.1879) |
          (aweish > 0.1112) | (ndvi < -0.2) | (ndwi_leaves > 1.0))

    if FILTER_UABS:
        ws = ws & ~((aweinsh <= -0.03) | (dbsi > 0))

    return ws

def fai(b04,b07,b8a):
    # FAI = B07 - (B04 + (B8A - B04) * (783-665)/(865-665))
    return b07 - (b04 + (b8a - b04) * ((783 - 665) / (865 - 665)))

def ndci(b04,b05):
    return (b05 - b04) / (b05 + b04 + 1e-6)

def chlorophyll_from_ndci(ndci_arr):
    # Polinomio del script
    return 826.57*ndci_arr**3 - 176.43*ndci_arr**2 + 19.0*ndci_arr + 4.071

def ndvi(b04,b08):
    return (b08 - b04) / (b08 + b04 + 1e-6)

def ndwi(b03,b08):
    return (b03 - b08) / (b03 + b08 + 1e-6)

### Procesamiento de `.tiffs` en base a su fecha

In [5]:
def process_one(base_path: Path):
    """Devuelve diccionario de métricas agregadas y mapas intermedios."""
    (b02,b03,b04,b05,b07,b08,b8a,b11,b12), scl, dm = load_stack(base_path)

    valid = (dm > 0) & (~np.isin(scl, list(SCL_BAD)))

    # máscara de agua del script
    water = water_mask(b02,b03,b04,b08,b11,b12) & valid

    # índices
    FAIv  = fai(b04,b07,b8a)
    NDCIv = ndci(b04,b05)
    chl   = chlorophyll_from_ndci(NDCIv)

    NDVIv = ndvi(b04,b08)
    NDWIv = ndwi(b03,b08)

    def agg(masked):
        vals = masked[water]
        return dict(mean=float(np.nanmean(vals)) if vals.size else np.nan,
                    p90 =float(np.nanpercentile(vals,90)) if vals.size else np.nan,
                    p99 =float(np.nanpercentile(vals,99)) if vals.size else np.nan)

    metrics = {
        "chl_mean":  agg(chl)["mean"],
        "chl_p90":   agg(chl)["p90"],
        "FAI_mean":  agg(FAIv)["mean"],
        "NDCI_mean": agg(NDCIv)["mean"],
        "NDVI_mean": agg(NDVIv)["mean"],
        "NDWI_mean": agg(NDWIv)["mean"],
        "water_px":  int(water.sum())
    }
    maps = {"water": water, "chl": chl, "FAI": FAIv, "NDCI": NDCIv, "NDVI": NDVIv, "NDWI": NDWIv}
    return metrics, maps


### Recorrer ambos lagos y armar series (convirtiendo a arreglos de numpy)

In [6]:
rows = []
maps_cache = {}

for lake in LAKES:
    lake_folder = BASE / lake
    bases = list_date_bases(lake_folder)
    print(f"{lake}: {len(bases)} fechas encontradas")
    for date, base in bases:
        try:
            metrics, maps = process_one(base)
            rows.append({"lake": lake, "date": pd.to_datetime(date), **metrics})
            maps_cache.setdefault(lake, {})[date] = maps
        except Exception as e:
            print(f"[warn] {lake} {date}: {e}")

df = pd.DataFrame(rows).sort_values(["lake","date"]).reset_index(drop=True)
df.head()


atitlan: 29 fechas encontradas
amatitlan: 29 fechas encontradas


,lake,date,chl_mean,chl_p90,FAI_mean,NDCI_mean,NDVI_mean,NDWI_mean,water_px
0,amatitlan,2025-02-07,23.214016,45.584991,0.001404,0.310398,-0.28148,0.574654,133219
1,amatitlan,2025-02-10,NaN,NaN,NaN,NaN,NaN,NaN,0
2,amatitlan,2025-02-25,NaN,NaN,NaN,NaN,NaN,NaN,0
3,amatitlan,2025-02-27,31.240353,60.351524,0.020709,0.351973,0.07352,0.245286,109021
4,amatitlan,2025-03-02,NaN,NaN,NaN,NaN,NaN,NaN,0


### Llenado de los datos faltantes imputándolos con mediana

In [7]:
df["date"] = pd.to_datetime(df["date"])

START = "2025-02-01"
END   = "2025-08-01"
TARGET_CAL = pd.date_range(START, END, freq="D")

METRICS = ["chl_mean","FAI_mean","NDCI_mean","NDVI_mean","NDWI_mean"]

In [8]:
def complete_and_impute(df, metrics=METRICS, how="median"):
    """Rellena días faltantes por lago con la mediana o el promedio del lago (por métrica)."""
    filled = []
    for lake, g in df.groupby("lake", dropna=False):
        g = g.set_index("date").sort_index()

        re = g.reindex(TARGET_CAL)

        re["is_imputed_row"] = ~re.index.isin(g.index)

        re["lake"] = lake
        re.index.name = "date"

        if how == "median":
            fill_values = re[metrics].median(skipna=True)
        elif how == "mean":
            fill_values = re[metrics].mean(skipna=True)
        else:
            raise ValueError("how debe ser 'median' o 'mean'")

        for col in metrics:
            re[col] = re[col].fillna(fill_values[col])

        filled.append(re.reset_index())

    out = pd.concat(filled, ignore_index=True).sort_values(["lake","date"])
    return out

df_full = complete_and_impute(df, metrics=METRICS, how="median")

print(df_full.groupby("lake")[METRICS].apply(lambda x: x.isna().sum()))

           chl_mean  FAI_mean  NDCI_mean  NDVI_mean  NDWI_mean
lake                                                          
amatitlan         0         0          0          0          0
atitlan           0         0          0          0          0
